In [1]:
# to download the dataset
!gdown --id 1YLaRefVhcCZGuc-3_eqlpOxUQ0Iypq7J
# to unzip the file
!unzip train.zip

Downloading...
From: https://drive.google.com/uc?id=1YLaRefVhcCZGuc-3_eqlpOxUQ0Iypq7J
To: C:\Repositories\MachineLearning\week15\train.zip

  0%|          | 0.00/11.8M [00:00<?, ?B/s]
  4%|4         | 524k/11.8M [00:00<00:02, 3.83MB/s]
 18%|#7        | 2.10M/11.8M [00:00<00:01, 9.41MB/s]
 40%|###9      | 4.72M/11.8M [00:00<00:00, 14.8MB/s]
 53%|#####3    | 6.29M/11.8M [00:00<00:00, 13.2MB/s]
 71%|#######   | 8.39M/11.8M [00:00<00:00, 14.3MB/s]
 98%|#########7| 11.5M/11.8M [00:00<00:00, 18.3MB/s]
100%|##########| 11.8M/11.8M [00:00<00:00, 15.4MB/s]
'unzip'은(는) 내부 또는 외부 명령, 실행할 수 있는 프로그램, 또는
배치 파일이 아닙니다.


In [3]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# define model
def define_model():
    model = VGG16(include_top=False, input_shape=(224, 224, 3))
    # mark loaded layers as not trainable
    for layer in model.layers:
        layer.trainable = False
    # add new classifier layers
    flat1 = Flatten()(model.layers[-1].output)
    class1 = Dense(128, activation='relu', kernel_initializer='he_uniform')(flat1)
    output = Dense(1, activation='sigmoid')(class1)
    # define new model
    model = Model(inputs=model.inputs, outputs=output)
    # compile model
    opt = SGD(lr=0.001, momentum=0.9)
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
    return model

# run the test harness for evaluating a model
def run_test_harness():
    # define model
    model = define_model()
    # create data generator
    datagen = ImageDataGenerator(featurewise_center=True)
    # specify imagenet mean values for centering
    datagen.mean = [123.68, 116.779, 103.939]
    # prepare iterator
    train_it = datagen.flow_from_directory('train/',
                                           class_mode='binary', batch_size=64, target_size=(224,224))
    # fit model
    model.fit_generator(train_it, steps_per_epoch=len(train_it), epochs=5, verbose=1)
    # save model
    model.save('final_model_homework.h5')
    
# entry point, run the test harness
run_test_harness()

Found 999 images belonging to 2 classes.


C:\Users\wjsdu\anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/5
16/16 [==============================] - 18s 510ms/step - loss: 1.2658 - accuracy: 0.8308
Epoch 2/5
16/16 [==============================] - 3s 167ms/step - loss: 0.3022 - accuracy: 0.9520
Epoch 3/5
16/16 [==============================] - 3s 167ms/step - loss: 0.0675 - accuracy: 0.9870
Epoch 4/5
16/16 [==============================] - 3s 168ms/step - loss: 0.0204 - accuracy: 0.9940
Epoch 5/5
16/16 [==============================] - 3s 167ms/step - loss: 0.0052 - accuracy: 0.9980


In [4]:
from keras.models import load_model
import tensorflow as tf

# load saved model
model = load_model('final_model_homework.h5')

# convert Keras model to TFLite model
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# save converted model
with open('dandelion_vs_grass_class_model.tflite', 'wb') as f:
  f.write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\wjsdu\AppData\Local\Temp\tmp9iiak5bw\assets
